# Artificial Neural Networks and Deep Learning

---

## Homework 1: Minimal Working Example

To make your first submission, follow these steps:
1. Create a folder named `[2024-2025] AN2DL/Homework 1` in your Google Drive.
2. Upload the `training_set.npz` file to this folder.
3. Upload the Jupyter notebook `Homework 1 - Minimal Working Example.ipynb`.
4. Load and process the data.
5. Implement and train your model.
6. Submit the generated `.zip` file to Codabench.


## 🌐 Connect Colab to Google Drive

In [ ]:
from google.colab import drive

drive.mount('/gdrive')
%cd /gdrive/My Drive/AN2DL/Homework 1

## ⚙️ Import Libraries

In [ ]:
import numpy as np

import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl

np.random.seed(42)
tf.random.set_seed(42);

## ⏳ Load the Data

In [ ]:
data = dict(np.load("/gdrive/My Drive/[2024-2025] AN2DL/Homework 1/training_set.npz"))  # Load the training data here

In [ ]:
data["images"].shape, data["labels"].shape

In [ ]:
np.unique(data["labels"], return_counts = True) # 8 classes

In [ ]:
import matplotlib.pyplot as plt

# Assuming 'data' is already loaded as in the provided code

num_images_per_class = 2
image_shape = (96, 96, 3)

fig, axes = plt.subplots(4, 4, figsize=(20, 7))
j = 0
print(data["labels"])
for class_label in np.unique(data["labels"]):
  class_indices = np.where(data["labels"] == class_label)[0]
  selected_indices = np.random.choice(class_indices, size=min(num_images_per_class, len(class_indices)), replace=False)

  for i in selected_indices:
    image = data["images"][i]
    place = axes[j//4][j%4]
    place.imshow(image)
    place.axis('off')
    place.set_title(f"Class: {class_label}, Index: {i}")
    j+=1
plt.show()

In [ ]:
for class_label in np.unique(data["labels"]):
  class_indices = np.where(data["labels"] == class_label)[0]
  #print(class_label, data["images"][class_indices].min(), data["images"][class_indices].max())

In [ ]:
import numpy as np
from hashlib import md5
def find_identical_images(image_dataset, labels):
    """
    Find identical images in a dataset of images stored as NumPy arrays.

    Parameters:
        image_dataset (list or numpy.ndarray): List or array where each entry is an image in NumPy array format.

    Returns:
        duplicates (dict): A dictionary where keys are hash values, and values are lists of indices of identical images.
    """
    # Dictionary to store hashes and their corresponding image indices
    hash_dict = {}
    duplicates = {}

    for idx, image in enumerate(image_dataset):
        # Flatten the image and compute its hash
        image_hash = md5(image.tobytes()).hexdigest()

        # Check if the hash already exists
        if image_hash in hash_dict:
            if image_hash not in duplicates:
                duplicates[image_hash] = hash_dict[image_hash] # Start a list with the first duplicate
            duplicates[image_hash]["image_index"].append(idx)
            duplicates[image_hash]["image_label"].append(labels[idx][0])
        else:
            hash_dict[image_hash] = {"image_index": [idx], "image_label": [labels[idx][0]]}  # Store the index for the hash

    return duplicates

duplicates = find_identical_images(data["images"], data["labels"])

print(duplicates)

data["image_dubs"] = np.zeros(data["labels"].shape[0])

for hash_value, dubs in duplicates.items():
    labels = np.array(dubs["image_label"])
    same_label = np.all(labels == labels.flat[0])
    if same_label == False:
      data["image_dubs"][dubs["image_index"]] = 1
    else:
      data["image_dubs"][dubs["image_index"][1:]] = 1

In [ ]:
tot = 0
print("Number of duplicate images in each class:")
for class_label in np.unique(data["labels"]):
  class_indices = np.where(data["labels"] == class_label)[0]
  print(class_label, data["image_dubs"][class_indices].sum())
  tot += data["image_dubs"][class_indices].sum()
print(tot)

filtered_imgs = np.where(data["image_dubs"] == 0)[0]
print(f"len(filtered_imgs)={len(filtered_imgs)}")
new_data = {}
new_data["images"] = data["images"][filtered_imgs]
new_data["labels"] = data["labels"][filtered_imgs]
new_data["image_dubs"] = data["image_dubs"][filtered_imgs]

In [ ]:
find_identical_images(new_data["images"], new_data["labels"])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
class_names = [
    'Basophil', 'Eosinophil', 'Erythroblast', 'Immature granulocytes',
    'Lymphocyte', 'Monocyte', 'Neutrophil', 'Platelet'
]
labels = new_data["labels"].flatten()
# Calculate the total number of samples
total_samples = len(labels)
class_counts = np.bincount(labels)
# Calculate the percentage of each class
percentages = (class_counts / total_samples) * 100

# Plotting the pie chart
plt.figure(figsize=(8, 8))
plt.pie(percentages, labels=class_names, autopct='%1.1f%%', startangle=140, colors=plt.cm.Paired.colors)
plt.title('Percentage of Each Class in the Dataset')
plt.axis('equal')  # Equal aspect ratio ensures the pie chart is a circle.
plt.show()


In [ ]:
# prompt: find the average value of each channel in images of one class in a center frame of 36 pixels

def average_channel_values(images, labels, center_size=54):
    """
    Calculates the average value of each channel for images of a specific class
    in a central region of the image.

    Args:
        images: A NumPy array of images. Shape: (num_images, height, width, channels).
        labels: A NumPy array of labels corresponding to the images.
        target_class: The class label for which to calculate the averages.
        center_size: The size of the central region to consider.

    Returns:
        A NumPy array of shape (3,) containing the average values for each channel,
        or None if no images of the target class are found.
    """
    grayscale_images = 255 - (
        0.2989 * images[..., 0] +  # Red channel
        0.5870 * images[..., 1] +  # Green channel
        0.1140 * images[..., 2]    # Blue channel
    )

    avg_gray = 0
    for image_orig in grayscale_images:
        image = image_orig.copy()
        h, w = image.shape
        center_y = h // 2
        center_x = w // 2
        half_size = center_size // 2
        y_start = max(0, center_y - half_size)
        y_end = min(h, center_y + half_size)
        x_start = max(0, center_x - half_size)
        x_end = min(w, center_x + half_size)

        # image[y_start:y_end, x_start:x_end] = 0
        avg_gray += np.mean(image[y_start:y_end, x_start:x_end])

    return avg_gray / len(grayscale_images)


# Example usage (assuming 'new_data' and 'class_names' are defined as in your previous code):
for class_label in np.unique(new_data["labels"]):
    filtered_imgs = np.where(new_data["labels"] == class_label)[0]
    averages = average_channel_values(new_data["images"][filtered_imgs], new_data["labels"][filtered_imgs])
    if averages is not None:
        print(f"Average channel values: {averages}")
    else:
        print(f"No images found")

In [ ]:
import os
import cv2 as cv


masked_data = {}
masked_data["images"] = []
masked_data["labels"] = []
# Assuming 'new_data' is defined as in your provided code
for i, image in enumerate(new_data["images"]):
    img_orig = new_data["images"][i]
    img = cv.cvtColor(255-img_orig, cv.COLOR_BGR2GRAY)
    blur = cv.GaussianBlur(img,(5,5),0)
    ret3,th3 = cv.threshold(img, 118, 255,cv.THRESH_BINARY)

    kernel = np.ones((3, 3), np.uint8)

    th3_err = cv.erode(th3, kernel, iterations=1)
    kernel = np.ones((3, 3), np.uint8)
    th3_err = cv.dilate(th3_err, kernel, iterations=5)

    masked = cv.bitwise_and(img_orig, img_orig, mask=th3_err)
    masked_data["images"].append(masked)
    masked_data["labels"].append(new_data["labels"][i])

# np.savez("/content/drive/MyDrive/AN2DL/Homework 1/masked_data.npz", images=masked_data["images"],labels=masked_data["labels"])

In [ ]:
import cv2 as cv
from matplotlib import pyplot as plt

cv.imwrite('output.png', new_data["images"][2000])

img_orig = new_data["images"][2000]
img = cv.cvtColor(255-img_orig, cv.COLOR_BGR2GRAY)
# Otsu's thresholding after Gaussian filtering

blur = cv.GaussianBlur(img,(5,5),0)
ret3,th3 = cv.threshold(img, 118, 255,cv.THRESH_BINARY)

kernel = np.ones((3, 3), np.uint8)

th3_err = cv.erode(th3, kernel, iterations=1)
kernel = np.ones((3, 3), np.uint8)
th3_err = cv.dilate(th3_err, kernel, iterations=5)

masked = cv.bitwise_and(img_orig, img_orig, mask=th3_err)

plt.subplot(1,4,1),plt.imshow(img_orig)
plt.subplot(1,4,2),plt.imshow(img,'gray')
plt.subplot(1,4,3),plt.imshow(th3,'gray')
plt.subplot(1,4,4),plt.imshow(masked)
masked

In [ ]:

import math
colors = ("red", "green", "blue")
bins = np.arange(256)

class_names = [
    'Basophil', 'Eosinophil', 'Erythroblast', 'Immature granulocytes',
    'Lymphocyte', 'Monocyte', 'Neutrophil', 'Platelet'
]
class_labels = np.array([i for i in range(8)]).astype(np.uint8)

# create the histogram plot, with three lines, one for
# each color
plt.rc('axes', titlesize=8)  # Set the font size of subplot titles
plt.rc('axes', labelsize=8)
fig, axes = plt.subplots(8, 6, figsize=(14, 8))

column_labels = ['Example image', 'Average image', 'Hist for red', 'Hist for green', 'Hist for blue', 'Hist for gray']
for ax, label in zip(axes[0], column_labels):
    ax.set_title(label, fontsize=8)

row_labels = class_names
for ax, label in zip(axes[:, 0], row_labels):  # First column only
    ax.set_ylabel(label, fontsize=8, labelpad=10, rotation=45, ha='right', va='center')


axes = axes.flatten()

for label in class_labels:

  filtered_imgs = np.where(masked_data["labels"] == label)[0]
  images = np.array(masked_data["images"])[filtered_imgs]
  num_images = len(images)
  grayscale_images = 255 - (
    0.2989 * images[..., 0] +  # Red channel
    0.5870 * images[..., 1] +  # Green channel
    0.1140 * images[..., 2]    # Blue channel
  )

  axes[label*6].imshow(images[0])
  # axes[label*6].axis("off")
  axes[label*6+1].imshow(np.mean(images, axis=0).astype(np.uint8))
  # axes[label*6+1].axis("off")

  for channel_id, color in enumerate(colors):
    hist = np.zeros(len(bins), dtype=np.float32)

    for img in images:
        # Compute histograms for each channel
        histogram, bin_edges = np.histogram(
        img[:, :, channel_id], bins=256, range=(0, 256)
    )
        hist+=histogram

    # Average the histograms
    hist /= num_images
    axes[6*label+2+channel_id].plot(bins[1:], hist[1:], color=color)

  hist = np.zeros(len(bins), dtype=np.float32)
  for img in grayscale_images:
        histogram, bin_edges = np.histogram(
        img, bins=256, range=(0, 256)
    )
        hist+=histogram
  hist /= num_images
  axes[6*label+5].plot(bins[:-1], hist[:-1], color="k")

## Split the data

In [ ]:
import tensorflow as tf

from sklearn.model_selection import train_test_split
seed = 42
#X = (np.array(new_data['images'])/255).astype('float32')
X = (np.array(masked_data["images"])/255).astype('float32')
#y = np.array(labels)
y = np.array(masked_data["labels"])
y = tfk.utils.to_categorical(y)
print(f"shape of x: {X.shape}")
print(f"shape of y: {y.shape}")
print(f"max value in X: {X.max()}")

# Split data into train_val and test sets
#X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, random_state=seed, test_size=0.1, stratify=y)

# Further split train_val into train and validation sets
#X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, random_state=seed, test_size=0.1, stratify=y_train_val)
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=seed, test_size=0.1, stratify=y)
print("Training Data Shape:", X_train.shape)
print("Training Label Shape:", y_train.shape)
print("Validation Data Shape:", X_val.shape)
print("Validation Label Shape:", y_val.shape)
#print("Test Data Shape:", X_test.shape)
#print("Test Label Shape:", y_test.shape)

## Add augmentation

In [ ]:
#This is the function which will be applied to all images
augmentation = tfk.Sequential([
    tfkl.RandomFlip("horizontal_and_vertical"),
    tfkl.RandomRotation(0.3),
    #tfkl.RandomTranslation(0.05,0.05),
    tfkl.RandomZoom(0.1),
    tfkl.RandomBrightness(0.2, value_range=(0,1)),
    tfkl.RandomContrast(0.25),
], name='Augmentation')

#Plot images before augmentation
plt.subplot(2, 3, 1)
plt.imshow(X_train[0])
plt.subplot(2, 3, 2)
plt.imshow(X_train[1])
plt.subplot(2, 3, 3)
plt.imshow(X_train[2])

#We want to normalize so each category has the same amount of images
y_train_labels = np.argmax(y_train, axis=1) #from one-hot to label encoding
class_counts = np.bincount(y_train_labels)
print(f"class counts before augmentation {class_counts}")
#add images until each category has 3000
images_to_append = np.empty((13244,96,96,3))
labels_to_append = np.empty((13244,8))
append_at_index = 0
index = 0
while append_at_index < len(images_to_append):
  #index = index_app % len(X_train)
  image = X_train[index]
  if class_counts[y_train_labels[index]] < 3000:
    images_to_append[append_at_index] = image
    labels_to_append[append_at_index] = y_train[index]

    append_at_index += 1
    class_counts[y_train_labels[index]] += 1
  index += 1
  if index >= len(X_train):
    index = index %len(X_train)


#augmentation done in many steps because otherwise colab runs out of ram
steps = 40
for i in range(steps):
  start_ind = (i)*len(X_train)//steps
  end_ind = (i+1)*len(X_train)//steps
  X_train[start_ind:end_ind] = augmentation(X_train[start_ind:end_ind])
for i in range(steps):
  start_ind = (i)*len(images_to_append)//steps
  end_ind = (i+1)*len(images_to_append)//steps
  images_to_append[start_ind:end_ind] = augmentation(images_to_append[start_ind:end_ind])

print(f"class counts after augmentation {class_counts}")

plt.subplot(2, 3, 4)
plt.imshow(X_train[0])
plt.subplot(2, 3, 5)
plt.imshow(X_train[1])
plt.subplot(2, 3, 6)
plt.imshow(X_train[2])
plt.axis('off')

plt.subplot(2, 3, 2).set_title("Original images", fontsize=14)#, labelpad=10, rotation=45, ha='right', va='center')
plt.subplot(2, 3, 5).set_title("Augmented images", fontsize=14)#, labelpad=10, rotation=45, ha='right', va='center')

#.set_title("Title for first plot")
plt.show()


#this part is to save augmented_images.npz as a file
"""try:
  np.savez_compressed("/gdrive/My Drive/[2024-2025] AN2DL/Homework 1/augmented_images.npz", X_train=X_train, X_val=X_val, y_train=y_train, y_val=y_val)
  print("successful file save")
except Exception as e:
  print(f"an error occured: {e}")"""


In [ ]:
# Input shape for the model
input_shape = X_train.shape[1:]

# Output shape for the model
output_shape = y_train.shape[1]

print("Input Shape:", input_shape)
print("Output Shape:", output_shape)

# Number of training epochs
epochs = 20

# Batch size for training
batch_size = 128

# Learning rate: step size for updating the model's weights
learning_rate = 0.001

# Print the defined parameters
print("Epochs:", epochs)
print("Batch Size:", batch_size)
print("Learning Rare:", learning_rate)

## VGG19 PRETRAINED MODEL


In [ ]:
def build_model(
    input_shape=input_shape,
    output_shape=output_shape,
    learning_rate=learning_rate,
    seed=seed
):
  inputs = tfkl.Input(shape=input_shape)
  output = inputs

  #pretrained_model = tf.keras.applications.VGG19(include_top=False, input_shape=input_shape, weights='imagenet')
  pretrained_model = tf.keras.applications.VGG19(include_top=False, input_shape=input_shape, weights='imagenet')
  output = pretrained_model(output)

  output = tfkl.Flatten(name='flatten')(output)

  output = tfkl.Dense(output_shape, activation='softmax', name='last_dense')(output)

  pretrained_model.trainable = False
  # Connect input and output through the Model class
  model = tfk.Model(inputs=inputs, outputs=output, name='CNN')

  # Compile the model
  loss = tfk.losses.CategoricalCrossentropy()
  optimizer = tfk.optimizers.Adam(learning_rate) #switch from adam?
  metrics = ['accuracy']
  model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

  # Return the model
  return model, pretrained_model

# Define the patience value for early stopping
patience = 10

# Create an EarlyStopping callback
early_stopping = tfk.callbacks.EarlyStopping(
    monitor='val_accuracy',
    mode='max',
    patience=patience,
    restore_best_weights=True
)

# Store the callback in a list
callbacks = [early_stopping]

In [ ]:
# Build the model with specified input and output shapes
#model, pretrained_model = build_model()
model, pretrained_model = build_model()

# Plot the model architecture
#tfk.utils.plot_model(model, expand_nested=True, show_trainable=True, show_shapes=True, dpi=70)
tfk.utils.plot_model(model, show_shapes=True, show_layer_names=True)
# Display a summary of the model architecture
model.summary(expand_nested=True, show_trainable=True)

In [ ]:
history1 = model.fit(
    x=X_train,
    y=y_train,
    batch_size=batch_size,
    epochs=20,
    validation_data=(X_val, y_val),
    callbacks=callbacks
).history

#Fine tuning
for layer in model.layers:
  layer.trainable = True
learning_rate = learning_rate / 20


pretrained_model.trainable = False
for layer in pretrained_model.layers[len(pretrained_model.layers)-10:]:
  layer.trainable = True

# It's important to recompile your model after you make any changes
# to the `trainable` attribute of any inner layer, so that your changes
# are take into account
model.compile(optimizer=tfk.optimizers.Adam(learning_rate/10),  # Very low learning rate
              loss=tfk.losses.CategoricalCrossentropy(),#tfk.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

# Train end-to-end. Be careful to stop before you overfit!
history2 = model.fit(
  x=X_train,
  y=y_train,
  batch_size=batch_size,
  epochs=10,
  validation_data=(X_val, y_val),
  callbacks=callbacks
).history
#history = dict(history1) #full history
#history.update(history2)

# Calculate and print the final validation accuracy
#final_val_accuracy = round(max(history1['val_accuracy'])* 100, 2)
#max_binary_accuracy = round(max(history2['binary_accuracy'])* 100, 2)
#print(f'Max binary accuracy: {max_binary_accuracy}%')

# Save the trained model to a file with the accuracy included in the filename
#model_filename = 'CIFAR10_CNN_'+str(final_val_accuracy)+'.keras'
#model.save(model_filename)

In [ ]:
print(pretrained_model.layers)
for layer in pretrained_model.layers[len(pretrained_model.layers)-10:]:
  print(layer)

In [ ]:

# Plot training and validation loss
#plt.figure()
plt.plot(history1['loss']+history2['loss'], label='Training loss', alpha=.8)
plt.plot(history1['val_loss']+history2['val_loss'], label='Validation loss', alpha=.8)
plt.title('Loss')
plt.legend()
plt.grid(alpha=.3)

# Plot training and validation accuracy
plt.figure(figsize=(15, 2))
plt.plot(history1['accuracy'] + history2["accuracy"], label='Training accuracy', alpha=.8)
plt.plot(history1['val_accuracy'] + history2["val_accuracy"], label='Validation accuracy', alpha=.8)
plt.title('Accuracy')
plt.legend()
plt.grid(alpha=.3)
colors = ("red", "green", "blue")
bins = np.arange(256)

plt.show()
#model.save("/gdrive/My Drive/[2024-2025] AN2DL/Homework 1/weights.keras")

In [ ]:
# Load the saved model
#model = tfk.models.load_model('CIFAR10_CNN_92.75.keras')

# Display a summary of the model architecture
#model.summary(expand_nested=True, show_trainable=True)

# Plot the model architecture
#tfk.utils.plot_model(model, expand_nested=True, show_trainable=True, show_shapes=True, dpi=70)

In [ ]:
# Predict labels for the entire test set
predictions = model.predict(X_val, verbose=0)

# Display the shape of the predictions
print("Predictions Shape:", predictions.shape)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
import seaborn as sns
# Convert predictions to class labels
pred_classes = np.argmax(predictions, axis=-1)

# Extract ground truth classes
true_classes = np.argmax(y_val, axis=-1)
print(f"first predictions: {pred_classes[:5]}")
print(f"first true classes: {true_classes[:5]}")

# Calculate and display test set accuracy
accuracy = accuracy_score(true_classes, pred_classes)
print(f'Accuracy score over the test set: {round(accuracy, 4)}')

# Calculate and display test set precision
precision = precision_score(true_classes, pred_classes, average='weighted')
print(f'Precision score over the test set: {round(precision, 4)}')

# Calculate and display test set recall
recall = recall_score(true_classes, pred_classes, average='weighted')
print(f'Recall score over the test set: {round(recall, 4)}')

# Calculate and display test set F1 score
f1 = f1_score(true_classes, pred_classes, average='weighted')
print(f'F1 score over the test set: {round(f1, 4)}')

# Compute the confusion matrix
cm = confusion_matrix(true_classes, pred_classes)

# Combine numbers and percentages into a single string for annotation
annot = np.array([f"{num}" for num in cm.flatten()]).reshape(cm.shape)

# Plot the confusion matrix
plt.figure(figsize=(10, 8))
labels = [
    'Basophil', 'Eosinophil', 'Erythroblast', 'Immature granulocytes',
    'Lymphocyte', 'Monocyte', 'Neutrophil', 'Platelet'
]
sns.heatmap(cm.T, annot=annot, fmt='', xticklabels=list(labels), yticklabels=list(labels), cmap='Blues')
plt.xlabel('True labels')
plt.ylabel('Predicted labels')
plt.show()

In [ ]:
model.save('weights.keras')
# del model

## K-fold validation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score

def train_and_evaluate(data, labels, model_name, learning_rates, optimizers, k=5, epochs=30, batch_size=128):
    kf = KFold(n_splits=k, shuffle=True, random_state=42)

    best_combination = None
    best_val_loss = float('inf')

    results = []

    for optimizer_name in optimizers:
        for lr in learning_rates:
            print(f"Evaluating optimizer={optimizer_name}, learning_rate={lr}")

            train_losses, val_losses, train_accuracies, val_accuracies, test_accuracies, test_f1 = [], [], [], [], [], []

            for fold, (train_idx, val_idx) in enumerate(kf.split(data)):
                print(f"  Fold {fold + 1}/{k}")

                # Split the data into training and validation sets
                train_data, val_data = data[train_idx], data[val_idx]
                train_labels, val_labels = labels[train_idx], labels[val_idx]

                train_dataset = tf.data.Dataset.from_tensor_slices((train_data, train_labels))
                val_dataset = tf.data.Dataset.from_tensor_slices((val_data, val_labels))

                # Shuffle, batch, and prefetch for training
                train_dataset = train_dataset.shuffle(buffer_size=1000).batch(128).prefetch(tf.data.AUTOTUNE)
                val_dataset = val_dataset.batch(128).prefetch(tf.data.AUTOTUNE)

                # Create and compile the model
                if optimizer_name == 'adam':
                    optimizer = Adam(learning_rate=lr)
                elif optimizer_name == 'sgd':
                    optimizer = SGD(learning_rate=lr)
                elif optimizer_name == 'rmsprop':
                    optimizer = RMSprop(learning_rate=lr)
                else:
                    raise ValueError(f"Unsupported optimizer: {optimizer_name}")

                patience = 5

                # Create an EarlyStopping callback
                early_stopping = tfk.callbacks.EarlyStopping(
                    monitor='val_loss',
                    mode='max',
                    patience=patience,
                    restore_best_weights=True
                )

                # Store the callback in a list
                callbacks = [early_stopping]
                loss = tfk.losses.CategoricalCrossentropy()

                model = build_model()
                model.compile(optimizer=optimizer,
                              loss=loss,
                              metrics=['accuracy'])

                # Train the model
                history = model.fit(train_dataset,
                                    validation_data=val_dataset,
                                    callbacks=callbacks,
                                    epochs=epochs,
                                    batch_size=batch_size)

                # Record metrics
                train_losses.append(history.history['loss'])
                val_losses.append(history.history['val_loss'])
                train_accuracies.append(history.history['accuracy'])
                val_accuracies.append(history.history['val_accuracy'])

                predictions = model.predict(X_test, verbose=0)
                pred_classes = np.argmax(predictions, axis=-1)
                true_classes = np.argmax(y_test, axis=-1)
                test_accuracy = accuracy_score(true_classes, pred_classes)
                f1 = f1_score(true_classes, pred_classes, average='weighted')

                test_accuracies.append(test_accuracy)
                test_f1.append(f1)

                final_val_accuracy = round(max(history.history['val_accuracy'])* 100, 2)

                model_filename = f'{model_name}_lr_{lr}_opt_{optimizer_name}_{final_val_accuracy}_fold_{fold}.keras'
                model.save(model_filename)



            # Compute average metrics over all folds
            avg_train_loss = np.mean([loss[-1] for loss in train_losses])
            avg_val_loss = np.mean([loss[-1] for loss in val_losses])
            avg_train_accuracy = np.mean([acc[-1] for acc in train_accuracies])
            avg_val_accuracy = np.mean([acc[-1] for acc in val_accuracies])
            std_val_accuracy = np.std([acc[-1] for acc in val_accuracies])

            avg_test_accuracy = np.mean([acc[-1] for acc in test_accuracies])
            std_test_accuracy = np.std([acc[-1] for acc in test_accuracies])
            avg_test_f1 = np.mean([f1[-1] for f1 in test_f1])
            std_test_f1 = np.std([f1[-1] for f1 in test_f1])

            print(f"Avg Train Loss: {avg_train_loss:.4f}, Avg Val Loss: {avg_val_loss:.4f}")
            print(f"Avg Train Accuracy: {avg_train_accuracy:.4f}, Avg Val Accuracy: {avg_val_accuracy:.4f}, , Std Val Accuracy: {std_val_accuracy:.4f}")
            print(f"Avg Test Accuracy: {avg_test_accuracy:.4f}, Std Test Accuracy: {std_test_accuracy:.4f}, Avg Test F1: {avg_test_f1:.4f},  Std Test F1: {std_test_f1:.4f}")

            results.append({
                'optimizer': optimizer_name,
                'learning_rate': lr,
                'avg_val_loss': avg_val_loss,
                'avg_val_accuracy': avg_val_accuracy,
                'std_val_accuracy': std_val_accuracy,
                'avg_test_accuracy': avg_test_accuracy,
                'std_test_accuracy': std_test_accuracy,
                'avg_test_f1': avg_test_f1,
                'std_test_f1': std_test_f1
            })

            # Update best combination if validation loss improves
            if avg_val_loss < best_val_loss:
                best_val_loss = avg_val_loss
                best_combination = {'optimizer': optimizer_name, 'learning_rate': lr}

            # Plot training/validation loss and accuracy for the combination
            plt.figure(figsize=(12, 5))

            # Loss plot
            plt.subplot(1, 2, 1)
            for i in range(k):
                plt.plot(train_losses[i], label=f'Train Loss (Fold {i+1})', alpha=0.7)
                plt.plot(val_losses[i], label=f'Val Loss (Fold {i+1})', alpha=0.7)
            plt.title(f'Loss - Optimizer {model_name}: {optimizer_name}, LR: {lr}')
            plt.xlabel('Epochs')
            plt.ylabel('Loss')
            plt.legend()

            # Accuracy plot
            plt.subplot(1, 2, 2)
            for i in range(k):
                plt.plot(train_accuracies[i], label=f'Train Accuracy (Fold {i+1})', alpha=0.7)
                plt.plot(val_accuracies[i], label=f'Val Accuracy (Fold {i+1})', alpha=0.7)
            plt.title(f'Accuracy - Optimizer {model_name}: {optimizer_name}, LR: {lr}')
            plt.xlabel('Epochs')
            plt.ylabel('Accuracy')
            plt.legend()

            plt.tight_layout()
            plt.show()

    print("\nBest Combination:")
    print(best_combination)

# Example dataset
np.random.seed(42)

# Define learning rates and optimizers to evaluate
learning_rates = [0.01, 0.001, 0.0005]
optimizers = ['adam', 'sgd', 'rmsprop']

# Perform cross-validation and parameter tuning
train_and_evaluate(X_val, y_val, "Custom", learning_rates, optimizers, k=5, epochs=10, batch_size=128)


## 📊 Prepare Your Submission

To prepare your submission, create a `.zip` file that includes all the necessary code to run your model. It **must** include a `model.py` file with the following class:

```python
# file: model.py
class Model:
    def __init__(self):
        """Initialize the internal state of the model."""

    def predict(self, X):
        """Return a numpy array with the labels corresponding to the input X."""
```

The next cell shows an example implementation of the `model.py` file, which includes loading model weights from the `weights.keras` file and conducting predictions on provided input data. The `.zip` file is created and downloaded in the last notebook cell.

❗ Feel free to modify the method implementations to better fit your specific requirements, but please ensure that the class name and method interfaces remain unchanged.

In [ ]:
%%writefile model.py
import numpy as np

import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl
import cv2 as cv


class Model:
    def __init__(self):
        """
        Initialize the internal state of the model. Note that the __init__
        method cannot accept any arguments.

        The following is an example loading the weights of a pre-trained
        model.
        """
        self.neural_network = tfk.models.load_model('weights.keras')



    def predict(self, X):
        """
        Predict the labels corresponding to the input X. Note that X is a numpy
        array of shape (n_samples, 96, 96, 3) and the output should be a numpy
        array of shape (n_samples,). Therefore, outputs must not be one-hot
        encoded."""

        X = X/255

        preds = self.neural_network.predict(X)

        if len(preds.shape) == 2:
            preds = np.argmax(preds, axis=1)
        return preds

## Testing model.py

In [ ]:
import model as model_file
#following two lines are necessary to update model_file after changing it due...
#to jupyter notebook architecture
import importlib
importlib.reload(model_file)

final_model = model_file.Model()
X_test_corrected = X_val*255 #these are the same as the actual test set in range
final_pred = final_model.predict(X_test_corrected)
print(f"first predictions: {final_pred[:25]}")
print(f"first true classes: {true_classes[:25]}")

print(f"shape of the predictions: {final_pred.shape}")
final_accuracy = accuracy_score(true_classes, final_pred)
print(f"final accuracy: {final_accuracy}")

In [ ]:
from datetime import datetime
filename = f'submission_{datetime.now().strftime("%y%m%d_%H%M%S")}.zip'

# Add files to the zip command if needed
!zip {filename} model.py weights.keras

from google.colab import files
files.download(filename)